In [59]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import time
import re

# ---------  一些固定变量   -----------#
HEADERS = {
    'headers': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
URL = 'https://scholar.google.com/scholar'

articles = []
# URL='www.baidu.com'
PROXIES = {
    'http': 'socks5://127.0.0.1:1080',
    'https': 'socks5://127.0.0.1:1080'
}

class GoCrawler:
    @staticmethod
    def _get_response(url=URL, query=None, verify=True, proxies=None):
        response = requests.get(url=url, params=query, headers=HEADERS, verify=verify, proxies=proxies)
        return response if response.status_code == 200 else print(response.status_code)

    """
    fetch_article方法可以传入从数据库传过来的迭代出来的list
    """

    def fetch_article(self, query_list):
        start = 0
        query_dict = {
            'q': query_list,
            'start': start,
            'btnG': '',
            'hl': 'zh-CN'
        }
        if self._get_response(query=query_dict) is not None:
            soup = BeautifulSoup(self._get_response(query=query_dict,proxies=None).text, "lxml")
            total_pages = soup.find_all('td')[-2].get_text()
            print(total_pages)
            #limit = (int(total_pages) - 1) * 10  ## 这里要处理        
            limit = 10
            # print(limit)
            #articles = []  # 如果从外部传过来的lists进行迭代会有问题
            while limit >= start:
                query_dict = {
                'q': query_list,
                'start': start,
                'btnG': '',
                'hl': 'zh-CN'
                }
                soup = BeautifulSoup(self._get_response(query=query_dict).text, "lxml")
                for item in soup.find_all('div', class_='gs_r'):
                    article_title = item.h3.get_text().replace('[HTML][HTML] ', '')
                    ref_item = item.find('div', class_='gs_ggsd')
                    bref = item.find('div', class_='gs_rs')
                    article_bref = bref.get_text() if bref else None
                    article_ref = ref_item.a.get('href') if ref_item else None
                    citation = ''

                    #if self.find_keyword(article_ref,query_list):
                    article_list = [
                    article_title,
                    article_bref,
                    article_ref,
                    query_list,
                    citation]
                    articles.append(article_list)
                start += 10
        #print(articles)
        
#     def find_keyword(slef, article_bref, keyword1, keyword2):
#         if ((re.search(r"\."+keyword1,paragraph) is not None) | (re.search(r"\s"+keyword1+r"\s",paragraph) is not None)) & ((re.search(r"\."+keyword2,paragraph) is not None) | (re.search(r"\s"+keyword2+r"\s",paragraph) is not None)):
#             return True
#         return False
        

        return articles




In [71]:
import pymysql as ps
def push_gene_query():
    db_config = {
        'host': '10.10.172.41',
        'port': 3306,
        'user': 'xj',
        'password': 'xj3d',
        'db': 'dspdw_dev',
        'charset': 'utf8'
    }
    db = ps.connect(**db_config)
    # noinspection PyBroadException
    with db.cursor() as cursor:
        sql = "SELECT `gene_name`,`c_dot`,`p_dot` FROM nlp_alt_list"
        cursor.execute(sql)
        result = cursor.fetchall()
        return result


def query_generate(gene='APC', cdot='R653', pdot=None, is_quered=False):
    if pdot==None:
        return "\"{}\" AND (\"{}\" OR \"c.{}\")".format(gene, cdot, cdot)
    return "\"{}\" AND (\"{}\" OR \"c.{}\" OR \"p.{}\" OR \"{}\")".format(gene, cdot, cdot, pdot, pdot)



In [75]:
push_query = push_gene_query()

In [76]:
#------------需要翻墙------------------#
def main(push_query):
    for i,li in enumerate(push_query):
        query_list = query_generate(li[0],li[1])
        articles = GoCrawler().fetch_article(query_list)
        time.sleep(5)
        print(articles)


test = main((('AKT3', 'c.G670C', 'p.V224L'), ('AKT3', 'c.C721T', 'p.R241W')))
test
    
    
    

503
[]
503
[]


In [25]:
insert_to_db(main(push_query))

ConnectionError: HTTPSConnectionPool(host='scholar.google.com.hk', port=443): Max retries exceeded with url: /scholar?q=%22AKT3%22+AND+%28%22c.G670C%22+OR+%22c.c.G670C%22%29&start=0&btnG=&hl=zh-CN (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002882B204208>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))

In [ ]:
find_keyword(TEST[0][1],'APC','R653')